In [0]:
import pandas as pd
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import numpy as np

In [0]:
API_KEY_DONT_CHECK_IN_TO_GITHUB = 'KsMrjQpEkb3eSliv6LhR'
plotly.tools.set_credentials_file(username='javaxnerd', api_key=API_KEY_DONT_CHECK_IN_TO_GITHUB)

In [0]:
DATA_PATH = "/content/antibiotic"
USING_REEVAL = True
FAILURE_RATE_COL = "failurerate_reeval" if USING_REEVAL else "failurerate"
FAILURE_RATE_COL += "_x100"

EXPERIMENT = {"series" : ("constrained-combined.pkl",
                          "unconstrained-combined.pkl"),
              "objectives" : (FAILURE_RATE_COL,
                              "totalantibiotic",
                              "maximumconcentration"),
              "group-by" : "actual_len"
             }

X_AXIS = "totalantibiotic"
Y_AXIS = "maximumconcentration"
Z_AXIS = FAILURE_RATE_COL

OBJECTIVE_TITLES = {"failurerate"             : "Failure Rate",
                    "failurerate_reeval"      : "Failure Rate - Re-Eval",
                    "failurerate_x100"        : "Failure Rate (%)",
                    "failurerate_reeval_x100" : "Failure Rate (%) - Re-Eval",
                    "maximumconcentration"    : "Maximum Concentration (µg/ml)",
                    "totalantibiotic"         : "Total Antibiotic (µg/ml)",
                    "actual_len"              : "Treatment Duration (days)"}

X_LABEL = OBJECTIVE_TITLES[X_AXIS]
Y_LABEL = OBJECTIVE_TITLES[Y_AXIS]
Z_LABEL = OBJECTIVE_TITLES[Z_AXIS]

CONSTRAINED_LIMIT = 0.01 # In the range [0.0, 1.0]
MAX_FAILURE_RATES = (None, CONSTRAINED_LIMIT)

In [0]:
dataframes = [None] * len(EXPERIMENT["series"])

cols = ['x' + str(i) for i in range(10)]\
       + list(EXPERIMENT['objectives'])\
       + [EXPERIMENT['group-by']]
        
for i, series in enumerate(EXPERIMENT["series"]):
    load_from = DATA_PATH + "/" + series
    dataframes[i] = pd.read_pickle(load_from)
    dataframes[i] = dataframes[i][cols]

In [0]:
ALPHA = 1.0
OPACITY_UNCONSTRAINED = 0.5
OPACITY_CONSTRAINED = 0.6
SIZE_UNCONSTRAINED = 2.5
SIZE_CONSTRAINED = 4
COLORS = [None,
          'rgba(127, 0, 0, ' + str(ALPHA) + ')',
          'rgba(127, 127, 0, ' + str(ALPHA) + ')',
          'rgba(0, 127, 0, ' + str(ALPHA) + ')',
          'rgba(127, 0, 127, ' + str(ALPHA) + ')',
          'rgba(0, 0, 127, ' + str(ALPHA) + ')',
          'rgba(0, 127, 127, ' + str(ALPHA) + ')',
          'rgba(63, 0, 0, ' + str(ALPHA) + ')',
          'rgba(63, 63, 0, ' + str(ALPHA) + ')',
          'rgba(0, 63, 0, ' + str(ALPHA) + ')',
          'rgba(63, 0, 63, ' + str(ALPHA) + ')']

#'rgba(0, 0, 63)', 'rgba(63, 63, 63)'

In [0]:
def t(df):
    cols = {}
    for group in range(10):
        cols[group] = list(df['x' + str(group)])
    rv = []
    for i in range(len(df)):
        row = []
        for group in range(10):
            row.append(cols[group][i])
        row = [str(int(x)) for x in row]
        rv.append("[" + ", ".join(tuple(row)) + "]")
    return rv

In [0]:
def show_plot(df, colors, opacity, size, axis_labels=True):
    global OPACITY
    global SIZE

    group_by = EXPERIMENT['group-by']
    groups = list(int(x) for x in sorted(list(set(df[group_by]))))

    data = []
        
    assert len(COLORS) >= 1 + len(groups)
    for group in groups:

        df_select = df.loc[df[group_by] == group]

        xs = np.asarray(list(df_select[X_AXIS]))
        ys = np.asarray(list(df_select[Y_AXIS]))
        zs = np.asarray(list(df_select[Z_AXIS]))
        ts = np.asarray(t(df_select))

        trace = go.Scatter3d(
            x=xs,
            y=ys,
            z=zs,
            text=ts,
            name=str(group) + (" days" if group > 1 else " day"),
            mode='markers',
            marker=dict(
                size=size,
                color=COLORS[group],
                opacity=opacity
            )
        )

        data.append(trace)

    layout = go.Layout(
        scene=dict(
            xaxis=dict(
                title=X_LABEL if axis_labels else "",
                titlefont=dict(
                    family='Courier New, monospace',
                    size=18,
                    color='#7f7f7f'
                )
            ),
            yaxis=dict(
                title=Y_LABEL if axis_labels else "",
                titlefont=dict(
                    family='Courier New, monospace',
                    size=18,
                    color='#7f7f7f'
                )
            ),
            zaxis=dict(
                title=Z_LABEL if axis_labels else "",
                titlefont=dict(
                    family='Courier New, monospace',
                    size=18,
                    color='#7f7f7f'
                )
            )
        ),
        title="",
        margin=dict(
            l=0,
            r=0,
            b=0,
            t=0
        )
    )
    return go.Figure(data=data, layout=layout)

In [0]:
py.iplot(show_plot(dataframes[0],
                   COLORS,
                   OPACITY_UNCONSTRAINED,
                   SIZE_UNCONSTRAINED,
                   axis_labels=True), filename='unconstrained-combined')

In [0]:
py.iplot(show_plot(dataframes[1],
                   COLORS,
                   OPACITY_CONSTRAINED,
                   SIZE_CONSTRAINED,
                   axis_labels=True), filename='constrained-combined')

In [0]:
def show_plot_top(df, opacity, size):
    global OPACITY
    global SIZE

    #group_by = EXPERIMENT['group-by']
    #groups = list(int(x) for x in sorted(list(set(df[group_by]))))

    data = []

    xs = np.asarray(list(df[X_AXIS]))
    ys = np.asarray(list(df[Y_AXIS]))
    zs = np.asarray(list(df[Z_AXIS]))
    cols = zs
    ts = np.asarray(t(df))

    trace = go.Scatter(
        x=xs,
        y=ys,
        text=ts,
        #name=str(group) + (" days" if group > 1 else " day"),
        mode='markers',
        marker=dict(
            size=size,
            color=cols,
            opacity=opacity,
            colorbar=dict(
                title=Z_LABEL
            ),
        )
    )

    data.append(trace)

    layout = go.Layout(
        scene=dict(
            xaxis=dict(
                title=X_LABEL,
                titlefont=dict(
                    family='Courier New, monospace',
                    size=18,
                    color='#7f7f7f'
                )
            ),
            yaxis=dict(
                title=Y_LABEL,
                titlefont=dict(
                    family='Courier New, monospace',
                    size=18,
                    color='#7f7f7f'
                )
            ),
            #zaxis=dict(
            #    title=Z_LABEL if axis_labels else "",
            #    titlefont=dict(
            #        family='Courier New, monospace',
            #        size=18,
            #        color='#7f7f7f'
            #    )
            #)
        ),
        title="",
        margin=dict(
            l=0,
            r=0,
            b=0,
            t=0
        )
    )
    return go.Figure(data=data, layout=layout)

In [0]:
py.iplot(show_plot_top(dataframes[1],
                       OPACITY_CONSTRAINED,
                       SIZE_CONSTRAINED * 2),
         filename='constrained-combined-top')